In [46]:
import os
import glob
import logging
import cmocean
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
import netCDF4

In [77]:
figdir = "./figures/"
datafilelist = sorted(glob.glob("./*/*nc"))

In [70]:
domain = (-15.89, -15.279, 27.69, 28.2)
enlarge = 0
m = Basemap(projection='merc',
            llcrnrlon=domain[0] - enlarge, llcrnrlat=domain[2] - enlarge,
            urcrnrlon=domain[1] + enlarge, urcrnrlat=domain[3] + enlarge,
            lat_ts=0.5 * (domain[1] + domain[3]), resolution='f')

In [62]:
def load_lst_canary(datafile):
    with netCDF4.Dataset(datafile, "r") as nc:
        time = nc.variables["time"][0]
        timeunits = nc.variables["time"].units
        date = netCDF4.num2date(time, timeunits)

        lon = nc.variables["lon"][:]
        lat = nc.variables["lat"][:]
        goodlon = (lon >= domain[0]) & (lon <= domain[1])
        goodlat = (lat >= domain[2]) & (lat <= domain[3])
        lon = lon[goodlon]
        lat = lat[goodlat]
        lst = nc.get_variables_by_attributes(standard_name="surface_temperature")[0][0,goodlat, goodlon]
        
        # Convert to Celsius
        lst -= 273.15
        
        # Correct positions
        deltalon = abs(lon[1] - lon[0])
        deltalat = abs(lat[1] - lat[0])
        lon -= deltalon /2.
        lat += deltalat /2.
        
    return lon, lat, lst, date

In [84]:
def plot_lst_canary(lon, lat, lst, date, figname=None):
    llon, llat = np.meshgrid(lon, lat)
    plt.figure()
    m.pcolormesh(llon, llat, lst, 
                 latlon=True, cmap=cmocean.cm.thermal)
    m.drawcoastlines(linewidth=.2)
    plt.colorbar()
    plt.title(date, fontsize=16)
    if figname is not None:
        plt.savefig(figname, dpi=300, bbox_inches="tight")
    # plt.show()
    plt.close()

In [85]:
for d in datafilelist:
    fname = os.path.basename(d).replace(".nc", ".png")
    figname = os.path.join(figdir, fname)
    lon, lat, lst, date = load_lst_canary(d)
    plot_lst_canary(lon, lat, lst, date, figname)

/home/ctroupin/Software/PythonEnvs/Diva-python3.6/lib/python3.6/site-packages/mpl_toolkits/basemap/__init__.py:3413: MatplotlibDeprecationWarning: The ishold function was deprecated in version 2.0.
  b = ax.ishold()
/home/ctroupin/Software/PythonEnvs/Diva-python3.6/lib/python3.6/site-packages/mpl_toolkits/basemap/__init__.py:3422: MatplotlibDeprecationWarning: axes.hold is deprecated.
    See the API Changes document (http://matplotlib.org/api/api_changes.html)
    for more details.
  ax.hold(b)
